

# CS-456, Fall 2024, Modern Cyber Security
---

## TLS (Transport Layer Security) and HTTPS (Secure HyperText Transfer Protocol)
### Due by:


Deliverable: \[28 points\] <font color =red> Thursday, Dec 9, 11:59pm </font>

### Setup
---
All of the python libraries and command dependencies to successfully run this lab should already be installed and available on the CS lab machines.  This jupyter notebook can be run on a CS lab machine and you can use SSH tunnels to view the notebook on your local laptop (more details on [CS SNA web page](https://sna.cs.colostate.edu/software/jupyter/)).

If you are using the lab machines, take heed of the following:

In order to use both flask (anaconda-distributed) and cryptography (python3.8 site package) in lab machines we need following two exports exactly in this format, since python should use system-installed-python3.8's cryptography package before it reaches to anaconda distributed openssl. The anaconda/bin has to be appended at the end of PATH.


>`export PYTHONPATH="/usr/lib64/python3.8/site-packages:/usr/local/anaconda/lib/python3.8/site-packages:$PYTHONPATH"`

>`export PATH=$PATH:/usr/local/anaconda/bin`

This project requires you to call api from certificate authority server and repository server that are hosted on a lab machine. Due to this, whenevr you are calling any api from these servers, that must be done from a lab machine.

### References
you can look into these links for help with this assignment:

1.   [*Exploring HTTPS with Python*](  https://realpython.com/python-https/)
2.   [OPENSSL Cheat Sheet](https://www.freecodecamp.org/news/openssl-command-cheatsheet-b441be1e8c4a/)
3.   [X509 Certificates Reference](https://cryptography.io/en/latest/x509/reference/#x-509-reference)








<a id='deploy_web_server'></a>
## 1. Create and deploy a simple FLASK web server
---
In this section you are going to create a simple web server using FLASK.  We can then use `wget`, `curl` or simply use a web browser to fetch the contents of the web page.  


Use SHIFT-EXECUTE to execute the following code cell.  This first cell will simply write the python code to a file named `simple-server.py`.  

**Next, use a new terminal window (or tab) to run the program by typing `python simple-server.py`.  Leave the program running and come back to this notebook.**

*Note: you may have to `pip install flask` if it is not installed in your system.  The lab systems should already have it installed.  Do this installation in a separate terminal window.*

In [1]:
%%writefile simple-server.py

# web-server.py
# secret message about squirrels!

from flask import Flask

#Put your secret message
SECRET_MESSAGE = "secret message"
app = Flask(__name__)

@app.route("/")
def get_secret_message():
    return SECRET_MESSAGE

if __name__ == "__main__":
    app.run(port=4567)

Writing simple-server.py


Let's make sure the server is running. Go to a web browser and enter the following URL:

> `http://localhost:4567`

You should get the following super-exciting response to your HTTP request:

<div>
<img src="attachment:image.png" width="600">
</div>

You can also use `wget` and `curl` commands to fetch the data from your tiny web server.  Execute the following shell commands in the next two cells using SHIFT-EXECUTE. In both cases the web server should return the secret message.  

In [ ]:
! wget -qO- localhost:4567

In [ ]:
! curl http://localhost:4567

Run one of the cells again, but this time capture the network traffic on localhost using wireshark.  Your wireshark data should look like the screen capture shown below.  Notice that anyone listening in can see the secret message.  This is definitely not a good state of affairs...

----


<div>
<img src="attachment:image.png" width="700">
</div>

**You should stop the web server now by using CTL-C in the terminal window.  We will start it up again later with TLS enabled.**

<a id='create_certificates'></a>

## Task 1 [5 points]
---


1.   Get the simple server running on your machine and Submit a screenshot of the network traffic on localhost while using wireshark in the below cell. <font color='purple'> [2 point]



<img src="images/Task1.png" width="75%" />

2.   Exlain why you could see the the secret mesage? <font color='purple'> [3 points]

Using normal http, everything is sent in plaintext. So anyone can packet sniff and find this data.


<a id='create_certificates'></a>

## 2. Create the x.509 Certificates needed by TLS
---


x.509 certificates are digital documents that serve as electronic passports in the world of internet security. They are fundamental components of TLS (Transport Layer Security), which is the protocol that ensures secure communication over the internet.

TLS (Transport Layer Security) is a cryptographic protocol designed to provide secure communication between computer systems, particularly for web browsing, email, instant messaging, and other data transfers. When you see "https://" in your browser's address bar, it means TLS is being used to protect your connection.

x.509 certificates work within TLS by:
1. Verifying the identity of websites (authentication)
2. Enabling encrypted communication (confidentiality)
3. Ensuring data hasn't been tampered with (integrity)

These certificates contain:
- The website's public key
- Information about the website (domain name, organization, etc.)
- Digital signature from a trusted Certificate Authority (CA)
- Validity period

How many CAs are there on planet Earth? There are hundreds of CAs throughout the globe. Your web browser keeps a list of trusted CA's in your web browser's trust-store. In Firefox you can view the list of trusted CAs by going to Preferences, Security & Privacy, View Certificates. There are similar mechanisms in other browsers.

A sample screen-shot showing just a few of the trusted CAs in Firefox is shown below:

<img src="http://cs.colostate.edu/~cs456/images/TLS_CA_List.png" width='700px' />

<a id='create_web_certificate'> </a>
### 2.1. Create Your Web Server Certificate
----

A Certificate Signing Request (CSR) is a formal request to a Certificate Authority (CA) to create a digital certificate. Think of it like applying for a passport:
- You fill out an application with your personal information (CSR contains your server's information)
- You provide proof of identity (CSR contains your public key)
- You submit the application to the passport office (CSR is sent to CA)
- The passport office verifies and issues your passport (CA verifies and issues certificate)

The CSR contains:
- Your public key
- Your organization's information
- Domain name(s) the certificate will secure
- Contact information

There are several steps required to generate your site's web server certificate:

1. Create a Certificate Signing Request (CSR) which contains your public key: This is like filling out the information for your visa.
2. Send the CSR to a "Trusted Third Party (TTP)" Certificate Authority (CA): This is like sending your information into a visa application office.
3. The CA will verify your information:  CAs use a variety of verification techniques.
4. The CA  then approves your Public Key by digitally signing your CSR. This is equivalent to the TTP signing your visa.
5. Issue the verified Public Key certificate: This is equivalent to you receiving your visa in the mail.


<a id='deploy_web_server'></a>
## 2.1.A. Create a Certificate Signing Request (CSR)

----

<a id='create_CSR'> </a>
#### Create a Certificate Signing Request (CSR)

The first step to your web server becoming trusted is for you to generate a Certificate Signing Request (CSR). In the real world, the CSR would be sent to an actual Certificate Authority like Verisign or Let’s Encrypt. In this lab assignment, We have created our own CA which you will be using.

### Make sure to use your name or student ID in your certificate
* For "organizational unit name", use your own name so the TAs can verify you did your own work.

### CSR uses PUBLIC KEY associated with PRIVATE KEY internally, make sure to save your PRIVATE KEY

In [10]:
# Complete the following code to generate private key and CSR
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes

# Generate a private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# Get the public key
public_key = private_key.public_key()

# Save private key to file
with open("server-private-key.key", "wb") as f:
    f.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Save public key to file
with open("server-public-key.pem", "wb") as f:
    f.write(public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

# Create CSR builder and add subject information
builder = x509.CertificateSigningRequestBuilder()
builder = builder.subject_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, "localhost"),
    x509.NameAttribute(NameOID.ORGANIZATIONAL_UNIT_NAME, "Aidan Michalos"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "CSU"),
    x509.NameAttribute(NameOID.COUNTRY_NAME, "US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Colorado"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, "Fort Collins")
]))

# Add Subject Alternative Names (SAN) extension
builder = builder.add_extension(
    x509.SubjectAlternativeName([
        x509.DNSName("localhost")
    ]),
    critical=False
)

# Sign the CSR with your private key
csr = builder.sign(
    private_key,
    hashes.SHA256(),
    default_backend()
)

# Save CSR to file
with open("CSR.csr", "wb") as f:
    f.write(csr.public_bytes(serialization.Encoding.PEM))


<a id='deploy_web_server'></a>
## 2.1.B. Verify the CSR

----
Next, make sure the CSR got created properly. Execute the following cell to view the contents of the CSR. If you don't like what you see, just re-execute the python code/OPENSSL request to re-create the CSR

In [5]:
! openssl req -text -noout -verify -in CSR.csr

'openssl' is not recognized as an internal or external command,
operable program or batch file.



<a id='deploy_web_server'></a>
## 2.2. Certificate Authority Server

----

We have created our own certificate authrity server. Like any other trusted CA, this will sign your CSR and create a public key certificate for the public key you sent in your CSR.

N0te: Make sure you have the right private key stored before sending your CSR for signing. 


## 2.2.A. Activate account

----
For each student, there is an inactive account created in our CA server. You need to use your `eid` as username and `csu id` as password.

Use the following URL to access the activation api on our CA server:
> `https://10.1.44.154:10200/active-account?username=your_eid&password=your_csu_id`

This will activate your account and return your `account id` in the response. This id will be used in later communications.
In case you forget your account`id`, you can call the same api again. You will get a response like this:



<div border=1>
<img src="attachment:image.png" width="700">
</div>

For future use you have to use the Id after the prefix string `‘SID#’`. In above case my Id is 96.


<a id='deploy_web_server'></a>
## 2.2.B. Submit the CSR to be digitally signed by the CA and get your X509 Certificate

----
In the real world, the CA would make sure that the information in your CSR are correct, like you owned invykta.com (or whatever you called your web-site) and ask you to prove it in various ways.

So when you call the CA server api to sign the CSR, it will verify your CSR and respond with x509 publick key certificate signed by the CA. So, you need to save the x509 certificate of your server you recieved from the CA in your machine.

Complete the python code below to call the CA server and get your x509 server public key certificate. This cell will write the python code to a file named `sign-csr.py`, so you need to run the code in a terminal. **In the submission this code should be completed.**



In [11]:
%%writefile sign-csr.py

import requests
from cryptography import x509
from cryptography.hazmat.primitives import serialization

# Load The CSR content
csr_file = open("CSR.csr", "rb")
csr_content = x509.load_pem_x509_csr(csr_file.read())

# URL of the API endpoint
url = "https://10.1.44.154:10200/sign-csr?sid=53"  # id (53)

# Sending the request with csr_content as form data
response = requests.get(url, data={'csr_content': csr_content.public_bytes(serialization.Encoding.PEM)}, verify=False)

# Check if the request was successful
if response.status_code == 200:
    filename = "signed-server-public-key.pem"
    
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Certificate saved to {filename}")

else:
    print(f"Error with status code: {response.status_code}")
    print(response.text)

Overwriting sign-csr.py


Next, use a new terminal window (or tab) to run the program by typing: `python sign-csr.py`. This will generate a server certificate file in the specified directory. Make sure your CSR.csr file is in accessable directory.

## Task 2 [19 Points]

----


1. Activating your account through calling the CA server api. Submit a screenshot of the response in the below cell. <font color='purple'> [2 points]


<img src="images/Task2-response.png" width="75%" />

2. Take a screenshot of the response of CSR verification command (in 2.1.B) of your CSR that you generated in 2.1.A. <font color='purple'> [10 points]

<img src="images/Task2-CLI.png" width="75%" />

3. Use our CA api to send your csr to the CA to get your signed x509 public key certificate. Our CA server will store all the information passed through this api call. So apart from completing the sign-csr.py script and calling the api,  no further action is required. The code should save your x509 certificate in the specified file <font color='purple'> [5 points]


4.   To verify your server's certificate, execute the cell below. <font color='purple'> [2 points]

In [ ]:
# make sure the new server certificate file has been created and examine it in BASE64 and with openSSL. 
# Use the right certificate file name
! ls server*
! cat your_server_cert.pem
! openssl x509 -in your_server_cert.pem -text -noout

## 3  Test your TLS certificate with your web server

We have to make a minor adjustment to the FLASK web server to enable TLS/HTTPS. You will see this adjustment in the last line of code below.

Use SHIFT-EXECUTE to execute the following code cell to over-write `simple-server.py` with TLS enabled.  

**Next, use a new terminal window (or tab) to run the program by typing `python simple-server.py`.  In the code below the web server will read the private key from `'server-private-key.key'` and the public key from your certificate file `'server-public-key.pem'`.**


**Leave the program running and come back to this notebook.**

In [12]:
%%writefile simple-server.py

from flask import Flask

SECRET_MESSAGE = "Secret Message"
app = Flask(__name__)

@app.route("/")
def get_secret_message():
    return SECRET_MESSAGE

if __name__ == "__main__":
    #Please check whether the key and cert names are correct
    app.run(port=4567, ssl_context=( 'signed-server-public-key.pem', 'server-private-key.key') )  #ssl_context=('cert.crt', 'key.pem'))

Overwriting simple-server.py


Now bring up a web browser and enter the following into the URL bar:

>`http://localhost:4567`

This command __should fail__ to bring up the web page because it didn't specify HTTPS.  Try again with the following, and capture the web traffic with wireshark.

>`https://localhost:4567`

your browser may not trust the certificate yet, so you need to go through the browser options to "view the certificate" and take a screenshot to submit to Canvas.  

Tell your browser that you "accept the risk" and proceed to view your simple web page with HTTPS.

Examine the wireshark output and you will notice that the TLS handshake has occurred and the data has been encrypted. submit a screenshot of the wireshark highlighting the encrypted server response.


## Task 3 [4 points]
----
1. submit screenshot of your signed certificate used to access the web server in browser view <font color='purple'> [2 points]


<img src="images/Task3-https.png" height="75%"/>

2. submit screenshot showing the TLs handshake and encrypted trafic in  wireshark <font color='purple'> [2 points]



<img src="images/Task3-wire.png" height="75%"/>

---
## <font color =blue> Deliverable \[28 points\]:  Thursday, Dec 9, 11:59pm </font>
1. Complete all the tasks above, task 1 to task 3. In the code cell below each task, submit your answers.
2. Make sure you are maintaining proper directory for all the additional files.
3. Zip all the required files along with this notebook. Name the notebook and the zip file as `your_eid_D1`  and submit to canvas.
---
